In [2]:
!ls /home/jupyter/20000360102458359xu/MarketScan/MDCD

'2014 v20'  '2016 v10'	'2018 v30'  '2020 v20'	'2022 v10'
'2015 v20'  '2017 v10'	'2019 v20'  '2021 v10'	'2023 v10'


### Naming Pattern of the parquet files

In [3]:
!ls /home/jupyter/20000360102458359xu/MarketScan/MDCD/2014\ v20/PARQUET

medicaid_a_2014_v20_0_0_0.snappy.parquet
medicaid_a_2014_v20_0_1_0.snappy.parquet
medicaid_a_2014_v20_0_2_0.snappy.parquet
medicaid_a_2014_v20_0_3_0.snappy.parquet
medicaid_a_2014_v20_0_4_0.snappy.parquet
medicaid_a_2014_v20_0_5_0.snappy.parquet
medicaid_a_2014_v20_0_6_0.snappy.parquet
medicaid_a_2014_v20_0_7_0.snappy.parquet
medicaid_d_2014_v20_0_0_0.snappy.parquet
medicaid_d_2014_v20_0_0_1.snappy.parquet
medicaid_d_2014_v20_0_0_2.snappy.parquet
medicaid_d_2014_v20_0_0_3.snappy.parquet
medicaid_d_2014_v20_0_1_0.snappy.parquet
medicaid_d_2014_v20_0_1_1.snappy.parquet
medicaid_d_2014_v20_0_1_2.snappy.parquet
medicaid_d_2014_v20_0_1_3.snappy.parquet
medicaid_d_2014_v20_0_2_0.snappy.parquet
medicaid_d_2014_v20_0_2_1.snappy.parquet
medicaid_d_2014_v20_0_2_2.snappy.parquet
medicaid_d_2014_v20_0_2_3.snappy.parquet
medicaid_d_2014_v20_0_3_0.snappy.parquet
medicaid_d_2014_v20_0_3_1.snappy.parquet
medicaid_d_2014_v20_0_3_2.snappy.parquet
medicaid_d_2014_v20_0_3_3.snappy.parquet
medicaid_d_2014_

In [4]:
import sys
import os
import re
import json
import pandas as pd
from collections import defaultdict



In [5]:

def parse_parquet_files(data_path: str, year: str, prefix: str) -> dict:
    """
    Group parquet files by data source from a specific year
    
    Args:
        folder_path: Path to the folder containing parquet files from different data tables
        year: Year extracted from the folder path
        prefix: Database prefix to look for (e.g., 'ccae', 'mdcr')
        
    Returns:
        - grouped_files: Dictionary with structure {prefix_table_code_year: [list of files]}
    """
    
    # Dictionary to store grouped files
    grouped_files = defaultdict(list)
    
    # Get all parquet files in the folder matching the prefix
    file_list = [os.path.join(data_path, f) for f in os.listdir(data_path) 
                if f.endswith('.snappy.parquet') and f.startswith(prefix)]
    
    # Group files by table code
    for filepath in file_list:
        filename = os.path.basename(filepath)
        # Get the table code (character after the prefix)
        
        table_code = filename[len(prefix):len(prefix)+1]

        if table_code == "_":
            # take the next character as table code
            table_code = filename[len(prefix)+1:len(prefix)+2]
        # Group key is prefix + table_code
        group_key = f"{prefix}_{table_code}_{year}"
        grouped_files[group_key].append(filepath)
    
    # Sort files within each group
    for group in grouped_files:
        grouped_files[group].sort()
    
    return dict(grouped_files)

In [11]:

path_template = "/home/jupyter/20000360102458359xu/MarketScan/MDCD/{year_folder}/PARQUET"
years = !ls /home/jupyter/20000360102458359xu/MarketScan/MDCD

# Filter valid years that start with "20xx"
valid_years = [y for y in years if re.match(r"^20\d{2}", y)]
  # List of prefixes

# Dictionary to store combined results
medicaid_paths = defaultdict(list)

# Iterate over all combinations of data_path and prefix
for year_folder in valid_years:
    result = parse_parquet_files(data_path=path_template.format(year_folder=year_folder), 
                                 year= re.search(r"\b20\d{2}\b", year_folder).group(),
                                 prefix='medicaid')
    
    # Merge dictionaries by appending values to lists
    for key, file_list in result.items():
        medicaid_paths[key].extend(file_list)

# Convert to a normal dictionary if needed
medicaid_data_paths = dict(medicaid_paths)



### sanity check for table and year count 

In [13]:
# sanity check for table and year count
import pandas as pd

sanity_check = pd.DataFrame({"keys": list(medicaid_data_paths.keys())})
sanity_check["table"] = sanity_check["keys"].str.extract(r'medicaid_(.)_\d{4}$')
sanity_check["year"] = sanity_check["keys"].str.extract(r'(\d{4})$')  # Extract year

# group by table and count how many years for each table
sanity_check.groupby("table").size().reset_index(name="year_count")


,table,year_count
0,a,10
1,d,10
2,f,10
3,i,10
4,l,10
5,o,10
6,s,10
7,t,10


### Write the data into a csv file

In [14]:
# write medicaid_data_paths to a json file
with open("medicaid_data_paths_dict.json", "w") as f:
    json.dump(medicaid_data_paths, f)


In [15]:
medicaid_data_paths

{'medicaid_a_2014': ['/home/jupyter/20000360102458359xu/MarketScan/MDCD/2014 v20/PARQUET/medicaid_a_2014_v20_0_0_0.snappy.parquet',
  '/home/jupyter/20000360102458359xu/MarketScan/MDCD/2014 v20/PARQUET/medicaid_a_2014_v20_0_1_0.snappy.parquet',
  '/home/jupyter/20000360102458359xu/MarketScan/MDCD/2014 v20/PARQUET/medicaid_a_2014_v20_0_2_0.snappy.parquet',
  '/home/jupyter/20000360102458359xu/MarketScan/MDCD/2014 v20/PARQUET/medicaid_a_2014_v20_0_3_0.snappy.parquet',
  '/home/jupyter/20000360102458359xu/MarketScan/MDCD/2014 v20/PARQUET/medicaid_a_2014_v20_0_4_0.snappy.parquet',
  '/home/jupyter/20000360102458359xu/MarketScan/MDCD/2014 v20/PARQUET/medicaid_a_2014_v20_0_5_0.snappy.parquet',
  '/home/jupyter/20000360102458359xu/MarketScan/MDCD/2014 v20/PARQUET/medicaid_a_2014_v20_0_6_0.snappy.parquet',
  '/home/jupyter/20000360102458359xu/MarketScan/MDCD/2014 v20/PARQUET/medicaid_a_2014_v20_0_7_0.snappy.parquet'],
 'medicaid_d_2014': ['/home/jupyter/20000360102458359xu/MarketScan/MDCD/2014